In [1]:
import warnings

import entwiner
import networkx as nx

# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
central_bounds = (-122.3611, 47.5994, -122.2773, 47.6183)

In [3]:
G = entwiner.DiGraphDBView(path="../artifacts/unweaver/graph.gpkg")

In [4]:
# Create subgraph from bounding box
G2 = entwiner.DiGraphDB.create_graph(path="../artifacts/unweaver/graph.betweenness.gpkg")
edges = []
for e in G.network.edges.intersects(*central_bounds):
    e.pop("fid")
    edges.append(e)
G2.network.edges.write_features(edges)

In [5]:
with G2.network.gpkg.connect() as conn:
    column_exists = False
    for row in conn.execute("PRAGMA table_info(edges)"):
        if row["name"] == "wheelchair_weight":
            column_exists = True
    if not column_exists:
        conn.execute("ALTER TABLE edges ADD COLUMN wheelchair_weight DOUBLE")

In [6]:
# Embed wheelchair weights

# Set weight to 1000 (1 km) because centrality function can't handle None values (inf)

with G2.network.gpkg.connect() as conn:
    conn.execute("""
        UPDATE edges
           SET wheelchair_weight = length
    """)
    conn.execute("""
        UPDATE edges
           SET wheelchair_weight = 1000
         WHERE (incline IS NOT NULL) AND (ABS(incline) > 0.1)
    """)
    conn.execute("""
        UPDATE edges
           SET wheelchair_weight = 1000
         WHERE (curbramps IS NOT NULL) AND (curbramps != 1)
    """)

In [7]:
# Calculate and save betweenness centrality for different cost functions
cost_fun = lambda u, v, d: d["length"]
betweenness = nx.algorithms.edge_betweenness_centrality(G2, weight="length")
for (u, v), centrality in betweenness.items():
    G2[u][v]["betweenness"] = centrality

In [9]:
# Calculate and save betweenness centrality for different cost functions
cost_fun_wc = lambda u, v, d: d["wheelchair_weight"]

betweenness_wc = nx.algorithms.edge_betweenness_centrality(G2, weight="wheelchair_weight")
for (u, v), centrality in betweenness_wc.items():
    G2[u][v]["betweenness_wc"] = centrality